In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype 
import implicit
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
import scipy
from gensim.models import Word2Vec
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Msi-1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Msi-1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Msi-1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
ratings = pd.read_csv('ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [5]:
# Зададим бинарные оценки 2 - книга понравилась, 1 - не понравилась, если задать, как в задании, то книги, которые не читали и книги, 
# которые не понравились, не будут отличаться
ratings.loc[ratings['rating'] <= 3, 'rating'] = 1
ratings.loc[ratings['rating'] > 3, 'rating'] = 2
ratings

,user_id,book_id,rating
0,1,258,2
1,2,4081,2
2,2,260,2
3,2,9296,2
4,2,2318,1
...,...,...,...
5976474,49925,510,2
5976475,49925,528,2
5976476,49925,722,2
5976477,49925,949,2


In [7]:
# Пронумеруем прочитанные книги
ratings = ratings.sort_values(by=['user_id'])
number = []
n = 1
for i in range(0, len(ratings)):
    if i == 0:
        number.append(n)
        continue
    if ratings.iloc[i,0] != ratings.iloc[i-1,0]:
        n = 1
        number.append(n)
        continue
    else:
        n += 1
        number.append(n)
        continue
ratings['number'] = number
ratings  

,user_id,book_id,rating,number
0,1,258,2,1
999485,1,140,1,2
999486,1,869,2,3
999487,1,2679,1,4
999488,1,1310,2,5
...,...,...,...,...
5404305,53424,5500,2,129
5404304,53424,4214,2,130
5404302,53424,2032,2,131
5404318,53424,41,2,132


In [9]:
# Посчитаем количество прочитанных
count = ratings.groupby('user_id').max().reset_index()[['user_id', 'number']]
count.rename(columns={'number': 'count'}, inplace=True)
ratings = ratings.merge(count, how='left')
ratings

,user_id,book_id,rating,number,count
0,1,258,2,1,117
1,1,140,1,2,117
2,1,869,2,3,117
3,1,2679,1,4,117
4,1,1310,2,5,117
...,...,...,...,...,...
5976474,53424,5500,2,129,133
5976475,53424,4214,2,130,133
5976476,53424,2032,2,131,133
5976477,53424,41,2,132,133


In [11]:
# Переведем номера прочитанных книг в доли
ratings['fractions'] = ratings['number'] / ratings['count']
ratings

,user_id,book_id,rating,number,count,fractions
0,1,258,2,1,117,0.008547
1,1,140,1,2,117,0.017094
2,1,869,2,3,117,0.025641
3,1,2679,1,4,117,0.034188
4,1,1310,2,5,117,0.042735
...,...,...,...,...,...,...
5976474,53424,5500,2,129,133,0.969925
5976475,53424,4214,2,130,133,0.977444
5976476,53424,2032,2,131,133,0.984962
5976477,53424,41,2,132,133,0.992481


In [13]:
# Получим матрицу для трейна
train = ratings[ratings.fractions <= 0.7].drop(columns=['number', 'count', 'fractions'])
user_index = train.user_id.unique()
books_index = train.book_id.unique()

rows = train['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes
cols = train['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes

matrix = sparse.csr_matrix((train.rating, (rows, cols)), shape=(len(user_index), len(books_index)))
matrix = matrix.toarray()
matrix.shape

(53424, 10000)

In [15]:
# Получим матрицу для теста
test = ratings[ratings.fractions > 0.7].drop(columns=['number', 'count', 'fractions'])
# test.to_csv('test for Collabaration.csv', index=False)
user_index1 = test.user_id.unique()
books_index1 = test.book_id.unique()

rows1 = test['user_id'].astype(CategoricalDtype(categories=user_index1)).cat.codes
cols1 = test['book_id'].astype(CategoricalDtype(categories=books_index1)).cat.codes

matrix1 = sparse.csr_matrix((test.rating, (rows1, cols1)), shape=(len(user_index1), len(books_index1)))
matrix1 = matrix1.toarray()
matrix1.shape

(53424, 10000)

In [17]:
# Функция, которая подсчитывает метрику AP@K
def average_precision_at_k(recommendations, positive_books, k):
    found = 0
    accumulated_precision = 0
    if len(positive_books) > 0:
        for i, book in enumerate(recommendations):
            if book in positive_books:
                found += 1
                precision_at_k = found / (i + 1)
                accumulated_precision += precision_at_k
        
        average_precision = accumulated_precision / min(k, len(positive_books))
    else:
        average_precision = 0
    
    return average_precision

In [128]:
# случайный список из 100 пользовательских индексов
user_random = np.random.randint(matrix.shape[0], size=10)

In [23]:
# Обучим модель с базовыми парамтерами
model = implicit.als.AlternatingLeastSquares(factors=100, iterations=45, num_threads=4)

model.fit(sparse.csr_matrix(matrix))

  0%|          | 0/45 [00:00<?, ?it/s]

In [130]:
# Посчитаем метрики модели Implicit
baseline_ap_list = []

for user in tqdm.tqdm(user_random):
    recomended_books = set(model.recommend(user, sparse.csr_matrix(matrix)[user], N=10)[0])
    user_books = set(np.where(matrix[user]>0)[0])
    recommendations = recomended_books - user_books
    positive_books = []
    
    for recommendation in recommendations:
        if recommendation in np.where(matrix1[user]>1)[0]:
            positive_books.append(recommendation)

    AP = average_precision_at_k(recommendations,  positive_books, 10)
    baseline_ap_list.append(AP) 
            
mean_ap_at_10 = sum(baseline_ap_list) / len(baseline_ap_list)

print("Средняя метрика AP@10 для модели Implicit:", mean_ap_at_10)   

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.74s/it]

Средняя метрика AP@10 для модели Implicit: 0.0125


In [31]:
book_tags = pd.read_csv('book_tags.csv')
book_tags

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716
...,...,...,...
999907,33288638,21303,7
999908,33288638,17271,7
999909,33288638,1126,7
999910,33288638,11478,7


In [33]:
tags = pd.read_csv('tags.csv')
tags

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-
...,...,...
34247,34247,Ｃhildrens
34248,34248,Ｆａｖｏｒｉｔｅｓ
34249,34249,Ｍａｎｇａ
34250,34250,ＳＥＲＩＥＳ


In [35]:
tag_merge = book_tags.merge(tags, how='left')
tag_merge.drop(columns=['tag_id'], inplace=True)
tag_merge

,goodreads_book_id,count,tag_name
0,1,167697,to-read
1,1,37174,fantasy
2,1,34173,favorites
3,1,12986,currently-reading
4,1,12716,young-adult
...,...,...,...
999907,33288638,7,neighbors
999908,33288638,7,kindleunlimited
999909,33288638,7,5-star-reads
999910,33288638,7,fave-author


In [37]:
# Выберем наиболее популярные и подходящие теги для создания фичей
best_tags = [ 'fiction', 'fantasy',
              'young-adult', 'classics',  'romance',
              'mystery', 'non-fiction', 'historical-fiction', 
              'science-fiction', 'sci-fi', 'paranormal', 'kindle', 'contemporary',
              'horror',
              'urban-fantasy', 
              'adult', 'childrens', 'graphic-novels', 'thriller',
              'vampires', 'adventure', 'history', 'dystopian',
              'humor', 'novels', 'chick-lit',
              'paranormal-romance',
              'magic', 'crime', 'comics', 'memoir', 
              'adult-fiction', 'biography',
              'supernatural', 'manga', 'philosophy', 'teen',
              'science', 
              'suspense', 'poetry', 
              'realistic-fiction', 'short-stories', 'drama']

In [39]:
tag_merge = tag_merge[tag_merge.tag_name.isin(best_tags)]
tag_merge

,goodreads_book_id,count,tag_name
1,1,37174,fantasy
4,1,12716,young-adult
5,1,9954,fiction
12,1,3374,magic
13,1,2408,childrens
...,...,...,...
999832,33288638,29,chick-lit
999842,33288638,21,fiction
999869,33288638,12,humor
999877,33288638,10,drama


In [41]:
tag_merge = pd.get_dummies(tag_merge, columns=['tag_name'], prefix='book', dtype='int64')
tag_merge

,goodreads_book_id,count,book_adult,book_adult-fiction,book_adventure,book_biography,book_chick-lit,book_childrens,book_classics,book_comics,book_contemporary,book_crime,book_drama,book_dystopian,book_fantasy,book_fiction,book_graphic-novels,book_historical-fiction,book_history,book_horror,book_humor,book_kindle,book_magic,book_manga,book_memoir,book_mystery,book_non-fiction,book_novels,book_paranormal,book_paranormal-romance,book_philosophy,book_poetry,book_realistic-fiction,book_romance,book_sci-fi,book_science,book_science-fiction,book_short-stories,book_supernatural,book_suspense,book_teen,book_thriller,book_urban-fantasy,book_vampires,book_young-adult
1,1,37174,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,12716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,1,9954,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,3374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1,2408,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999832,33288638,29,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999842,33288638,21,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999869,33288638,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999877,33288638,10,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
tag_merge.iloc[:,2:] = tag_merge.iloc[:,2:].multiply(tag_merge['count'], axis="index") 
tag_merge

,goodreads_book_id,count,book_adult,book_adult-fiction,book_adventure,book_biography,book_chick-lit,book_childrens,book_classics,book_comics,book_contemporary,book_crime,book_drama,book_dystopian,book_fantasy,book_fiction,book_graphic-novels,book_historical-fiction,book_history,book_horror,book_humor,book_kindle,book_magic,book_manga,book_memoir,book_mystery,book_non-fiction,book_novels,book_paranormal,book_paranormal-romance,book_philosophy,book_poetry,book_realistic-fiction,book_romance,book_sci-fi,book_science,book_science-fiction,book_short-stories,book_supernatural,book_suspense,book_teen,book_thriller,book_urban-fantasy,book_vampires,book_young-adult
1,1,37174,0,0,0,0,0,0,0,0,0,0,0,0,37174,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,12716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12716
5,1,9954,0,0,0,0,0,0,0,0,0,0,0,0,0,9954,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,3374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1,2408,0,0,0,0,0,2408,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999832,33288638,29,0,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999842,33288638,21,0,0,0,0,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999869,33288638,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999877,33288638,10,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
# Создадим фичи отражающие количество тегов встречающихся на одну книгу
tag_merge = tag_merge.groupby('goodreads_book_id')[tag_merge.iloc[:,2:].columns].sum().reset_index()
tag_merge

,goodreads_book_id,book_adult,book_adult-fiction,book_adventure,book_biography,book_chick-lit,book_childrens,book_classics,book_comics,book_contemporary,book_crime,book_drama,book_dystopian,book_fantasy,book_fiction,book_graphic-novels,book_historical-fiction,book_history,book_horror,book_humor,book_kindle,book_magic,book_manga,book_memoir,book_mystery,book_non-fiction,book_novels,book_paranormal,book_paranormal-romance,book_philosophy,book_poetry,book_realistic-fiction,book_romance,book_sci-fi,book_science,book_science-fiction,book_short-stories,book_supernatural,book_suspense,book_teen,book_thriller,book_urban-fantasy,book_vampires,book_young-adult
0,1,0,0,1742,0,0,2408,958,0,289,0,0,0,37174,9954,0,0,0,0,0,347,3374,0,0,465,0,744,673,0,0,0,0,334,0,0,0,0,484,0,600,0,547,0,12716
1,2,0,0,0,0,0,0,0,0,0,0,0,0,3441,972,0,0,0,0,0,371,253,0,0,447,0,764,688,0,0,0,0,213,199,0,0,0,473,0,613,0,562,0,865
2,3,0,0,2430,0,0,3828,1898,0,397,0,0,0,47478,13239,0,0,0,0,0,528,4302,0,0,657,0,1082,923,0,0,0,0,0,0,0,0,0,608,0,667,0,664,0,14984
3,5,0,0,1982,0,0,2960,1090,0,324,0,0,0,39330,10656,0,0,0,0,0,403,3618,0,0,526,0,845,720,0,0,0,0,0,0,0,0,0,520,0,566,0,584,0,12856
4,6,0,0,1892,0,0,2656,1032,0,301,0,0,0,38378,10249,0,0,0,0,0,387,0,0,0,493,0,826,702,0,0,0,0,0,0,0,0,0,491,0,581,0,568,0,12979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,31538647,0,0,0,0,0,43,0,0,0,0,0,0,893,233,0,0,0,0,0,205,97,0,0,0,0,6,11,0,0,0,0,0,0,0,0,247,10,0,6,0,10,0,193
9996,31845516,31,0,0,80,0,0,0,0,9,0,0,0,0,8,0,0,0,0,0,30,0,0,432,0,392,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,32075671,0,0,0,0,0,0,0,0,1130,56,31,0,0,696,0,0,0,0,0,74,0,0,0,0,0,39,0,0,0,0,373,37,0,0,0,0,0,0,168,0,0,0,2010
9998,32848471,55,14,0,0,23,0,0,0,8,0,14,0,0,11,0,0,0,0,29,56,0,0,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0


In [47]:
books = pd.read_csv('books.csv')
books

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,"Bayou Moon (The Edge, #2)",eng,4.09,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",eng,4.25,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,The Mauritius Command,eng,4.35,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,Cinderella Ate My Daughter: Dispatches from th...,eng,3.65,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [53]:
train_df = train[['user_id', 'book_id']]
train_y = train['rating']

In [55]:

# Создадим фичу отражающую средний бал поставленный пользователем
general = pd.concat([train_df, train_y], axis=1)
average = general.groupby('user_id').mean().reset_index()[['user_id', 'rating']]
average.rename(columns={'rating': 'rating_average'}, inplace=True)
train_df = train_df.merge(average, how='left')

train_df = train_df.merge(books[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
# Добавим фичи связанные с книгами
train_df = train_df.merge(tag_merge,  how='left')
# Добавим фичи связанные с пользователем
s = train_df.iloc[:,4:] > 0
s = s.astype('int64')
s['user_id'] = train_df.user_id
users_profiles = s.groupby('user_id').sum()
users_profiles.columns = ['user_'+name for name in list(users_profiles)]
train_df = train_df.merge(users_profiles, left_on='user_id', right_on='user_id', how='left')
train_df

,user_id,book_id,rating_average,goodreads_book_id,book_adult,book_adult-fiction,book_adventure,book_biography,book_chick-lit,book_childrens,book_classics,book_comics,book_contemporary,book_crime,book_drama,book_dystopian,book_fantasy,book_fiction,book_graphic-novels,book_historical-fiction,book_history,book_horror,book_humor,book_kindle,book_magic,book_manga,book_memoir,book_mystery,book_non-fiction,book_novels,book_paranormal,book_paranormal-romance,book_philosophy,book_poetry,book_realistic-fiction,book_romance,book_sci-fi,book_science,book_science-fiction,book_short-stories,book_supernatural,book_suspense,book_teen,book_thriller,book_urban-fantasy,book_vampires,book_young-adult,user_book_adult,user_book_adult-fiction,user_book_adventure,user_book_biography,user_book_chick-lit,user_book_childrens,user_book_classics,user_book_comics,user_book_contemporary,user_book_crime,user_book_drama,user_book_dystopian,user_book_fantasy,user_book_fiction,user_book_graphic-novels,user_book_historical-fiction,user_book_history,user_book_horror,user_book_humor,user_book_kindle,user_book_magic,user_book_manga,user_book_memoir,user_book_mystery,user_book_non-fiction,user_book_novels,user_book_paranormal,user_book_paranormal-romance,user_book_philosophy,user_book_poetry,user_book_realistic-fiction,user_book_romance,user_book_sci-fi,user_book_science,user_book_science-fiction,user_book_short-stories,user_book_supernatural,user_book_suspense,user_book_teen,user_book_thriller,user_book_urban-fantasy,user_book_vampires,user_book_young-adult
0,1,258,1.493827,1232,315,214,104,0,0,0,144,0,286,106,143,0,796,4560,0,2973,122,0,0,238,0,0,0,2613,0,296,0,0,0,0,0,355,0,0,0,0,0,230,0,311,0,0,0,73,68,16,9,18,6,49,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32
1,1,140,1.493827,6892870,426,323,111,0,0,0,0,0,486,2026,125,0,0,5675,0,0,0,0,0,637,0,0,0,4304,0,297,0,0,0,0,70,0,0,0,0,0,0,721,0,2897,0,0,0,73,68,16,9,18,6,49,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32
2,1,869,1.493827,2967752,0,0,0,0,0,0,0,0,0,0,0,0,0,238,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,0,34,0,0,0,0,0,0,0,0,0,0,0,46,73,68,16,9,18,6,49,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32
3,1,2679,1.493827,227593,21,41,0,0,12,0,0,0,60,0,6,0,0,699,0,6,0,0,0,12,0,0,0,0,0,31,0,0,0,0,7,10,0,0,0,0,0,0,0,0,0,0,0,73,68,16,9,18,6,49,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32
4,1,1310,1.493827,85301,60,60,0,0,0,0,0,0,121,0,0,0,0,1393,0,0,0,0,0,54,0,0,0,0,0,36,0,0,0,0,23,0,0,0,0,1307,0,0,0,0,0,0,0,73,68,16,9,18,6,49,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4159548,53424,518,1.956989,34268,0,0,581,0,0,1496,6034,0,0,0,0,0,3591,1541,0,0,0,0,0,172,172,0,0,0,0,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,35,26,67,1,10,58,61,4,13,5,15,9,61,92,4,40,27,4,26,79,35,3,0,17,10,72,13,2,6,11,19,33,19,2,20,7,11,6,35,8,10,3,77
4159549,53424,257,1.956989,13023,0,0,385,0,0,1254,6210,0,0,0,0,0,3840,2096,0,0,0,0,109,427,54,0,0,0,0,195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,474,35,26,67,1,10,58,61,4,13,5,15,9,61,92,4,40,27,4,26,79,35,3,0,17,10,72,13,2,6,11,19,33,19,2,20,7,11,6,35,8,10,3,77
4159550,53424,332,1.956989,65605,0,0,0,0,0,1542,0,0,0,0,0,0,109,397,0,0,0,0,0,129,422,0,0,0,0,198,0,0,0,0,0,0,57,0,64,0,0,0,65,0,0,0,149,35,26,67,1,10,58,61,4,13,5,15,9,61,92,4,40,27,4,26,79,35,3,0,17,10,72,13,2,6,11,19,33,19,2,20,7,11,6,35,8,10,3,77
4159551,53424,245,1.956989,2839,0,0,198,0,0,1500,1294,0,160,0,81,0,1299,1980,0,0,0,0,0,73,0,0,0,0,0,122,0,0,0,0,533,0,0,0,0,0,0,0,78,0,0,0,2245,35,26,

In [57]:
# Функция очистки текста
def text_cleaner(text):
    # Преобразование текста в нижний регистр
    text = text.lower()

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text)

    # Преобразование сокращений
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"'m", " am", text)

    # Удаление знаков препинания
    text = re.sub(r'[^\w\s]', '', text)

    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_text = [word for word in tokens if word.lower() not in stop_words]

    # Лемматизация слов
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]

    # Объединение лемматизированных слов обратно в текст
    cleaned_text = ' '.join(lemmatized_text)

    return cleaned_text

In [59]:
# Подготовим Название книг к векторизации
books['original_title'] = books['original_title'].astype('str').map(lambda texts : text_cleaner(texts))
sentences = [text.split() for text in books['original_title']]

# Векторизация
w2v_model = Word2Vec(sentences=sentences, min_count=1, vector_size=10)
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=10)
w2v_model.save('title_vecs.npy')
w2v_model = Word2Vec.load('title_vecs.npy')

title_vecs = []
for title in books['original_title']:
    
    
    vec = np.mean([w2v_model.wv[word] for word in title.split() if word in w2v_model.wv], axis=0)
    
    title_vecs.append(vec)
    

df = pd.DataFrame([pd.Series(x) for x in title_vecs])
df.columns = ['vectors_{}'.format(x+1) for x in df.columns]
df

C:\Users\Msi-1\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Msi-1\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,vectors_1,vectors_2,vectors_3,vectors_4,vectors_5,vectors_6,vectors_7,vectors_8,vectors_9,vectors_10
0,0.215745,-0.068878,0.114823,0.079119,0.066215,0.110119,0.244843,0.151871,-0.364145,-0.185276
1,0.222627,-0.043490,0.196419,-0.031872,0.242014,0.193523,0.333273,0.174723,-0.476040,-0.156300
2,0.164198,-0.056235,0.087686,-0.088165,0.116002,0.171117,0.201555,0.170529,-0.333893,-0.099072
3,0.138653,0.015644,0.101861,-0.021995,0.082889,0.099749,0.208436,0.043892,-0.218124,-0.090980
4,0.265790,-0.076852,0.242259,0.029596,0.311155,0.262219,0.278468,0.154070,-0.506533,-0.205333
...,...,...,...,...,...,...,...,...,...,...
9995,0.176875,-0.064125,0.146915,0.035440,0.074059,0.145721,0.174785,0.125552,-0.280334,-0.067359
9996,0.003644,0.008424,-0.007480,-0.010060,-0.016283,0.033213,0.119207,0.017081,-0.076787,-0.062725
9997,0.067894,0.051047,0.047389,0.020729,0.067844,0.035040,0.027991,0.056277,-0.012678,0.012049
9998,0.163597,-0.047796,0.122579,-0.004702,0.127034,0.121932,0.240361,0.136503,-0.320336,-0.120263


In [61]:
books = pd.concat([books, df], axis=1)
books

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,vectors_1,vectors_2,vectors_3,vectors_4,vectors_5,vectors_6,vectors_7,vectors_8,vectors_9,vectors_10
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,hunger game,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,0.215745,-0.068878,0.114823,0.079119,0.066215,0.110119,0.244843,0.151871,-0.364145,-0.185276
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,harry potter philosopher stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,0.222627,-0.043490,0.196419,-0.031872,0.242014,0.193523,0.333273,0.174723,-0.476040,-0.156300
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,0.164198,-0.056235,0.087686,-0.088165,0.116002,0.171117,0.201555,0.170529,-0.333893,-0.099072
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,kill mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,0.138653,0.015644,0.101861,-0.021995,0.082889,0.099749,0.208436,0.043892,-0.218124,-0.090980
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,great gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,0.265790,-0.076852,0.242259,0.029596,0.311155,0.262219,0.278468,0.154070,-0.506533,-0.205333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,bayou moon,"Bayou Moon (The Edge, #2)",eng,4.09,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,0.176875,-0.064125,0.146915,0.035440,0.074059,0.145721,0.174785,0.125552,-0.280334,-0.067359
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,mean ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",eng,4.25,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,0.003644,0.008424,-0.007480,-0.010060,-0.016283,0.033213,0.119207,0.017081,-0.076787,-0.062725
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,mauritius command,The Mauritius Command,eng,4.35,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...,0.067894,0.051047,0.047389,0.020729,0.067844,0.035040,0.027991,0.056277,-0.012678,0.012049
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,cinderella ate daughter dispatch frontlines ne...,Cinderella Ate My Daughter: Dispatches from th...,eng,3.65,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...,0.163597,-0.047796,0.122579,-0.004702,0.127034,0.121932,0.240361,0.136503,-0.320336,-0.120263


In [63]:
# Добавим в тренировочный датасет векторы названия книг и другие фичи из датасета с книгами 
train_df = train_df.merge(books[list(df.columns)+['book_id', 'average_rating', 'ratings_count', 'work_text_reviews_count', 'ratings_5']], left_on='book_id', right_on='book_id')

# Выполним окончательную очистку датасета
train_df.drop(columns=['user_id','book_id','goodreads_book_id'], inplace=True)
train_df.fillna(0, inplace=True)
train_df

,rating_average,book_adult,book_adult-fiction,book_adventure,book_biography,book_chick-lit,book_childrens,book_classics,book_comics,book_contemporary,book_crime,book_drama,book_dystopian,book_fantasy,book_fiction,book_graphic-novels,book_historical-fiction,book_history,book_horror,book_humor,book_kindle,book_magic,book_manga,book_memoir,book_mystery,book_non-fiction,book_novels,book_paranormal,book_paranormal-romance,book_philosophy,book_poetry,book_realistic-fiction,book_romance,book_sci-fi,book_science,book_science-fiction,book_short-stories,book_supernatural,book_suspense,book_teen,book_thriller,book_urban-fantasy,book_vampires,book_young-adult,user_book_adult,user_book_adult-fiction,user_book_adventure,user_book_biography,user_book_chick-lit,user_book_childrens,...,user_book_comics,user_book_contemporary,user_book_crime,user_book_drama,user_book_dystopian,user_book_fantasy,user_book_fiction,user_book_graphic-novels,user_book_historical-fiction,user_book_history,user_book_horror,user_book_humor,user_book_kindle,user_book_magic,user_book_manga,user_book_memoir,user_book_mystery,user_book_non-fiction,user_book_novels,user_book_paranormal,user_book_paranormal-romance,user_book_philosophy,user_book_poetry,user_book_realistic-fiction,user_book_romance,user_book_sci-fi,user_book_science,user_book_science-fiction,user_book_short-stories,user_book_supernatural,user_book_suspense,user_book_teen,user_book_thriller,user_book_urban-fantasy,user_book_vampires,user_book_young-adult,vectors_1,vectors_2,vectors_3,vectors_4,vectors_5,vectors_6,vectors_7,vectors_8,vectors_9,vectors_10,average_rating,ratings_count,work_text_reviews_count,ratings_5
0,1.493827,315,214,104,0,0,0,144,0,286,106,143,0,796,4560,0,2973,122,0,0,238,0,0,0,2613,0,296,0,0,0,0,0,355,0,0,0,0,0,230,0,311,0,0,0,73,68,16,9,18,6,...,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32,0.244487,-0.102752,0.084879,0.090706,0.203487,0.316876,0.255555,0.120296,-0.474284,-0.161874,4.24,263685,24652,157170
1,1.493827,426,323,111,0,0,0,0,0,486,2026,125,0,0,5675,0,0,0,0,0,637,0,0,0,4304,0,297,0,0,0,0,70,0,0,0,0,0,0,721,0,2897,0,0,0,73,68,16,9,18,6,...,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32,0.037651,-0.006150,0.021780,0.032520,0.091849,0.156300,0.121916,0.059955,-0.188402,-0.029826,4.20,443951,26766,226511
2,1.493827,0,0,0,0,0,0,0,0,0,0,0,0,0,238,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,0,34,0,0,0,0,0,0,0,0,0,0,0,46,73,68,16,9,18,6,...,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32,0.107713,-0.033893,0.079886,0.051287,0.064029,0.106179,0.151966,0.072569,-0.214970,-0.065664,3.73,101669,15335,35962
3,1.493827,21,41,0,0,12,0,0,0,60,0,6,0,0,699,0,6,0,0,0,12,0,0,0,0,0,31,0,0,0,0,7,10,0,0,0,0,0,0,0,0,0,0,0,73,68,16,9,18,6,...,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32,0.200437,-0.041164,0.073824,-0.008770,0.115842,0.189942,0.181368,0.066535,-0.226746,-0.135449,3.56,35677,2273,6758
4,1.493827,60,60,0,0,0,0,0,0,121,0,0,0,0,1393,0,0,0,0,0,54,0,0,0,0,0,36,0,0,0,0,23,0,0,0,0,1307,0,0,0,0,0,0,0,73,68,16,9,18,6,...,0,56,12,62,4,20,79,0,55,40,3,8,69,1,0,9,24,13,65,4,0,12,3,30,37,8,8,8,2,5,11,5,9,1,1,32,0.159219,-0.058198,0.006746,0.050562,0.033639,0.096730,0.139733,0.061944,-0.183021,-0.100505,4.11,60126,6978,25212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4159548,1.956989,0,0,581,0,0,1496,6034,0,0,0,0,0,3591,1541,0,0,0,0,0,172,172,0,0,0,0,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,35,26,67,1,10,58,...,4,13,5,15,9,61,92,4,40,27,4,26,79,35,3,0,17,10,72,13,2,6,11,19,33,19,2,20,7,11,6,35,8

In [65]:
# Обучим модель
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42, n_jobs=-1)

classifier.fit(train_df, train_y)

RandomForestClassifier(criterion='entropy', n_jobs=-1, random_state=42)

In [69]:
test_df = test[['user_id', 'book_id']]
test_y = test['rating']

In [ ]:
test_df = test_df.merge(average, how='left')
test_df = test_df.merge(books[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
test_df = test_df.merge(tag_merge,  how='left')

s1 = test_df.iloc[:,4:] > 0
s1 = s1.astype('int64')
s1['user_id'] = test_df.user_id

users_profiles1 = s1.groupby('user_id').sum()
users_profiles1.columns = ['user_'+name for name in list(users_profiles1)]
test_df = test_df.merge(users_profiles1, left_on='user_id', right_on='user_id', how='left')

test_df = test_df.merge(books[list(df.columns)+list(df1.columns)+['book_id', 'average_rating', 'ratings_count', 'work_text_reviews_count', 'ratings_5']], left_on='book_id', right_on='book_id')

test_df.drop(columns=['user_id','book_id','goodreads_book_id'], inplace=True)
test_df.fillna(0, inplace=True)
test_df

In [134]:
baseline_ap_list = []

for user in tqdm.tqdm(user_random):
    recomended_books = model.recommend(user, sparse.csr_matrix(matrix)[user], N=30)[0]
    predict = pd.DataFrame(columns=['user_id', 'book_id'])
    predict['book_id'] = recomended_books
    predict['user_id'] = user
    predict = predict.merge(average, how='left')
    predict = predict.merge(books[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
    
    
    predict = predict.merge(tag_merge,  how='left')
    
    s1 = predict.iloc[:,4:] > 0
    s1 = s1.astype('int64')
    s1['user_id'] = predict.user_id
    
    users_profiles1 = s1.groupby('user_id').sum()
    users_profiles1.columns = ['user_'+name for name in list(users_profiles1)]
    
    predict = predict.merge(users_profiles1, left_on='user_id', right_on='user_id', how='left')
    
    predict = predict.merge(books[list(df.columns)+['book_id', 'average_rating', 'ratings_count', 'work_text_reviews_count', 'ratings_5']], left_on='book_id', right_on='book_id')
    
    
    recomended_book = pd.DataFrame(columns=['book_id', 'predictions'])
    recomended_book['book_id'] = predict[['book_id']]
    predict.drop(columns=['user_id','book_id','goodreads_book_id'], inplace=True)
    
    predict.fillna(0, inplace=True)
    
    predictions = classifier.predict_proba(predict) 
    recomended_book['predictions'] = predictions[:,1]
    recomended_books = set(recomended_book.sort_values(by=['predictions'], ascending=False).iloc[:10,0])
    
    


        
    user_books = set(np.where(matrix[user]>0)[0])
    recommendations = recomended_books - user_books
    positive_books = []
    
    for recommendation in recommendations:
        if recommendation in np.where(matrix1[user]>1)[0]:
            positive_books.append(recommendation)

    AP = average_precision_at_k(recommendations,  positive_books, 10)
    baseline_ap_list.append(AP) 
            
mean_ap_at_10 = sum(baseline_ap_list) / len(baseline_ap_list)

print("Средняя метрика AP@10 для модели Implicit:", mean_ap_at_10)   

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.79s/it]

Средняя метрика AP@10 для модели Implicit: 0.016666666666666666


Качество модели удалось увеличить. Гибридный метод работает лучше, но он занмает значительно больше времени. Какой метод выбрать может зависеть от конкретной задачи.
